In [ ]:
%run "0.0 Data preparation.ipynb"

In [ ]:
positiveQuestions = indexedLikertQuestions.copy()
positiveQuestions = positiveQuestions[positiveQuestions.index % 2 == 1]
negativeQuestions = indexedLikertQuestions.copy()
negativeQuestions = negativeQuestions[negativeQuestions.index % 2 == 0]

## SUS score functions

In [ ]:
# transforms the agreement scores (1-5 Likert scale) into a 0-4 mark with 0 = bad and 4 = great
data20190603SUSNormalized = data20190603.copy()
for respondant in data20190603SUSNormalized.index:
    for question in indexedLikertQuestions:
        answer = data20190603SUSNormalized.loc[respondant, question]
        if pd.notna(answer):
            if question in negativeQuestions:            
                data20190603SUSNormalized.loc[respondant, question] = answer-1
            else:
                data20190603SUSNormalized.loc[respondant, question] = 5-answer

In [ ]:
def getSUSScore(question, game, data=data20190603SUSNormalized, printResult=True):
    _score = 0
    _maxScore = 0
    _gameQuestionData = data.loc[data[gameQuestion]==game, question]

    for _answer in _gameQuestionData:
        if pd.notna(_answer):
            _score += _answer
            _maxScore += 4

    _percentageScoreResult = _score*100/_maxScore
    if printResult:
        print("'" + question + "': " + game + " " + '{:1.1f}%'.format(_percentageScoreResult))
    return _percentageScoreResult

## Best SUS scores

In [ ]:
print("Best scores:")
for question in indexedLikertQuestions:
    bestGame = ""
    bestScore = 0
    for game in games:
        questionScore = getSUSScore(question, game, printResult=False)
        if questionScore > bestScore:
            bestGame = game
            bestScore = questionScore
    print("'" + question + "': " + bestGame + " " + '{:1.1f}%'.format(bestScore))

## Worst SUS scores

In [ ]:
print("Worst scores:")
for question in indexedLikertQuestions:
    worstGame = ""
    worstScore = 100
    for game in games:
        questionScore = getSUSScore(question, game, printResult=False)
        if questionScore < worstScore:
            worstGame = game
            worstScore = questionScore
    print("'" + question + "': " + worstGame + " " + '{:1.1f}%'.format(worstScore))

## SUS scores matrices

In [ ]:
allSUSScores = pd.DataFrame(index=indexedLikertQuestions, columns=games)
for game in games:
    for question in indexedLikertQuestions:
        allSUSScores.loc[question, game] = getSUSScore(question, game, printResult=False)

allSUSScores = allSUSScores.astype(float)

In [ ]:
# normalize using minimum

f = plt.figure()
ax = f.add_subplot(111)
h = sns.heatmap(allSUSScores,
            ax=ax,
            cmap=plt.cm.jet,
            vmin=0,
            vmax=100,
            square=True,
            cbar_kws = dict(use_gridspec=False,location="left")
           )
ax.yaxis.tick_right()
plt.yticks(rotation=0);

# manually moves x labels closer to their ticks
h.set_xticklabels(ax.get_xticklabels(), rotation=30)
indexer = 0
for label in ax.xaxis.get_majorticklabels():
    dx = (50 - indexer * 9)/72.; dy = 0/72. 
    offset = tr.ScaledTranslation(dx, dy, f.dpi_scale_trans)
    label.set_transform(label.get_transform() - offset)
    indexer+=1

In [ ]:
pd.concat((allSUSScores.idxmin(), allSUSScores.min()), 1)

In [ ]:
pd.concat((allSUSScores.idxmax(), allSUSScores.max()), 1)

## SUS scores variance

In [ ]:
def getVarianceOnQG(question, game, data=data20190603, printResult=True):
#    print("Q='"+question+"'; G="+game)
    gameQuestionData = data.loc[data[gameQuestion]==game, question]
    gameQuestionData = [int(v) for v in gameQuestionData if pd.notna(v)]
    return np.var(gameQuestionData)

In [ ]:
allVars = pd.DataFrame(index=indexedLikertQuestions, columns=games)
for game in games:
    for question in indexedLikertQuestions:
        allVars.loc[question, game] = getVarianceOnQG(question, game, printResult=False)

allVars = allVars.astype(float)

In [ ]:
f = plt.figure()
ax = f.add_subplot(111)
h = sns.heatmap(allVars,
            ax=ax,
            cmap=plt.cm.jet,
            vmin=0.4,
            vmax=2.2,
            square=True,
            cbar_kws = dict(use_gridspec=False,location="left")
           )
ax.yaxis.tick_right()
plt.yticks(rotation=0);

# manually moves x labels closer to their ticks
h.set_xticklabels(ax.get_xticklabels(), rotation=30)
indexer = 0
for label in ax.xaxis.get_majorticklabels():
    dx = (50 - indexer * 9)/72.; dy = 0/72. 
    offset = tr.ScaledTranslation(dx, dy, f.dpi_scale_trans)
    label.set_transform(label.get_transform() - offset)
    indexer+=1

In [ ]:
# recommendations:
#    - best game: which questions?
#    

In [ ]:
pd.concat((allVars.idxmin(), allVars.min()), 1)

In [ ]:
pd.concat((allVars.idxmax(), allVars.max()), 1)

### SUS Likert scale score variance

In [ ]:
saveFig = True
filenamePathStem = "bar-plots-with-error-bars/"

for question in indexedLikertQuestions:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    sns.barplot(x=gameQuestion, y=question, data=data20190603, ax = ax)
    ax.set_xticklabels(shortGameNames)
    plt.xlabel(question)
    plt.ylabel('Agreement - Likert scale')
    plt.ylim(1, 5)

    if saveFig:
        shortQuestion = shortQuestions.index[shortQuestions.values==question].values[0]
        filename = filenamePathStem + shortQuestion
        fig.savefig(filename)